In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dropout, Dense, Embedding, LSTM, Bidirectional
import pandas as pd
from pandas import get_dummies
import json
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

In [2]:
DATA_LOC = 'data/'
FILE_NAME = 'data_train.json'

# Load in JSON files and extract from JSON
reviews_json = open(DATA_LOC + FILE_NAME, 'r')
reviews = json.load(reviews_json)

FileNotFoundError: [Errno 2] No such file or directory: 'data/data_train.json'

In [ ]:
# Analyze the distribution of stars and reviews as seperate lists
texts = [review['text'] for review in reviews] # Features
stars = [review['stars'] for review in reviews] # Labels

In [ ]:
# Balance a dataset to remove over-represented samples

from collections import Counter
def balance_classes(texts, stars):
    freqs = Counter(stars)

    # the least common class is the maximum number we want for all classes
    maxAllowable = freqs.most_common()[-1][1]
    numAdded = {clss: 0 for clss in freqs.keys()}
    newStars = []
    newTexts = []
    for i, star in enumerate(stars):
        if numAdded[star] < maxAllowable:
            newStars.append(star)
            newTexts.append(texts[i])
            numAdded[star] += 1
    return newTexts, newStars

In [ ]:
balanced_texts, balanced_stars = balance_classes(texts, stars)
print("Distributions before and after dataset balancing\nNon-Balanced: {}\nBalanced: {}".format(Counter(stars), 
                                       Counter(balanced_stars)))

In [ ]:
# Look at distributions before and after dataset balancing
pre_labels, pre_values = zip(*Counter(stars).items())
post_labels, post_values = zip(*Counter(balanced_stars).items())
pre_indexes = np.arange(len(pre_labels))
post_indexes = np.arange(len(post_labels))
width = 1

plt.bar(pre_indexes, pre_values, width)
plt.bar(post_indexes, post_values, width)
plt.xticks(pre_indexes + width * 0.5, pre_labels)
plt.legend(['Pre-Balance', 'Post-Balance'])

plt.show()

In [ ]:
# Save pre and post balanced datasets as numpy.save()
from tempfile import TemporaryFile

preBal = TemporaryFile()
postBal = TemporaryFile()

np.save(preBal, stars)
np.save(postBal, balanced_stars)

In [ ]:
# Split files into training and testing sets
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectors = vectorizer.fit_transform(balanced_texts)
X_train, X_test, y_train, y_test = train_test_split(vectors, balanced_stars, test_size=0.33, random_state=42)

In [ ]:
#################### Train and Stack ############################

In [ ]:
def lstm_model(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'], num_units = 128, embedding_size = 16):
    model = tf.keras.Sequential()
    model.add(Embedding(input_dim = 10000, output_dim = embedding_size, input_length = 1050))
    model.add(Bidirectional(LSTM(num_units)))
    model.add(Dense(1, activation = 'softmax'))
    model.compile(optimizer = optimizer, loss = loss, metrics = metrics)
    return model

In [ ]:
from mlxtend.classifier import StackingClassifier

svm_clf = LinearSVC()
lstm_clf = lstm_model()
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[svm_clf, lstm_clf],
                         meta_classifier=lr)

In [ ]:
svm_clf.fit(X_train, y_train)
# lstm_clf.fit(X_train, y_train)
# sclf.fit(X_train, y_train)

In [ ]:
# Predict the trained and fitted model on the test set
predictions = svm_clf.predict(X_test)

In [ ]:
# Model's Accuracy Report of SVM
print(accuracy_score(y_test, predictions))

In [ ]:
# Classification Report of SVM


print(classification_report(y_test, predictions))

In [ ]:
# Confusion Matrix
# Rows: Predictions
# Cols: Correct class

print(confusion_matrix(y_test, predictions))